<a href="https://colab.research.google.com/github/OlegPozovnoy/MailRuMLHomeworks/blob/master/HW3/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/datasets/')

Drive already mounted at /datasets/; to attempt to forcibly remount, call drive.mount("/datasets/", force_remount=True).


In [5]:
!ls "/datasets/My Drive/datasets/"

players.pkl  results.pkl  tournaments.pkl


In [0]:
import pickle
import pandas as pd

In [0]:
tournaments = pd.read_pickle("/datasets/My Drive/datasets/tournaments.pkl")
#tournaments = pd.read_pickle("./tournaments.pkl")

tournaments_cleaned = [(v['id'], v['name'], v['dateStart'][0:4], v['type']['name'], v['questionQty']) for k,v in tournaments.items() if v['questionQty'] != None]
del tournaments

Создадим трейн и тест набор турниров.

In [8]:
train_id = [ v[0] for v in tournaments_cleaned if int(v[2])== 2019]
test_id = [v[0] for v in tournaments_cleaned if int(v[2])== 2020]

print(len(train_id))
print(len(test_id))

687
418


In [0]:
results = pd.read_pickle("/datasets/My Drive/datasets/results.pkl")
#results = pd.read_pickle("./results.pkl")
results_cleaned = {k: v for k,v in results.items() if k in train_id or k in test_id}

results_cleaned2 = {}
for k,v in results_cleaned.items():
  results_cleaned2[k] = []
  for s in v:
    if s.get('mask') != None:
      results_cleaned2[k].append((s.get('mask'),s.get('questionsTotal'), [k.get('player').get('id') for k in s.get('teamMembers')]))

del results
del results_cleaned

In [10]:
train_id = [id for id in train_id if id in results_cleaned2.keys() and len(results_cleaned2[id]) > 0]
test_id = [id for id in test_id if id in results_cleaned2.keys() and len(results_cleaned2[id]) > 0]

print(len(train_id))
print(len(test_id))

675
173


In [11]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import tqdm
import gc

# creating all questions and players set
all_words = []
for k,v in results_cleaned2.items():
  for team in v: 
    for p in range(len(team[0])):
        all_words.append(str(k)+'_'+str(p))
    for m in team[2]:
        all_words.append(m)

all_words = set(all_words)
print(len(all_words))

# creating storage for competetion results
all_results = {}
for s in all_words:
  all_results[s] = [0,0] # total rating and num of competetions


logistic = LogisticRegression(fit_intercept = False)

def process_tournament(id):
  task3 = []
  words = []
   
  for n in range(len(results_cleaned2[id])): # command id
    for p in range(len(results_cleaned2[id][n][0])): #answers
        quest = str(id)+'_'+str(p)
        team = n
        task3.append((quest, team,results_cleaned2[id][n][0][p]) )
        words.append(quest)
        words.append(team)

  if len(words)==0:
    return 
  
  words = set(words) # encoding questions and teams no

  idx_to_word = dict(enumerate(words))
  word_to_idx = dict([(y,x) for x,y in idx_to_word.items()])
  #preparing matrix
  X = np.zeros((len(task3),len(words)))
  Y = np.zeros(len(task3))
  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = 1 if str(task3[i][2]) == '1' else 0

  if len(set(Y)) <= 1:
    print ('<2 categories , skipping')
    return

  #print(X.shape)
  logistic.fit(X, Y)

  for s in range(len(logistic.coef_[0])):
    if isinstance(idx_to_word[s], int): ## teanm no
        #print('team '+ str(idx_to_word[s]))
        for player in results_cleaned2[id][idx_to_word[s]][2]:
            #print(player)
            all_results[player][1] +=1
            all_results[player][0] +=logistic.coef_[0][s]
    else: #question no
        #print('question '+ str(idx_to_word[s]))
        all_results[idx_to_word[s]][1] +=1
        all_results[idx_to_word[s]][0] +=logistic.coef_[0][s]

#process_tournament(1526)


104707


Пройдемся по всем турнирам и посчитаем рейтинг всех игроков и вопросов в заданном турнире, за неимением лучшего, усредним по каждому игроку рейтинг в турнире

In [12]:
import tqdm

for tourn_id in tqdm.tqdm(train_id):
  process_tournament(tourn_id)

 99%|█████████▊| 665/675 [01:54<00:00, 22.70it/s]

<2 categories , skipping


100%|██████████| 675/675 [02:45<00:00,  4.09it/s]


In [16]:
import statistics as stat

def is_player(t):
  return isinstance(t, int)

all_ranking = {}

for k,v in all_results.items():
  if v[1] != 0:
    all_ranking[k] = v[0]/v[1] 

blank_rank = stat.mean ([ v for k,v in all_ranking.items() if is_player(k)])
print(blank_rank)


-0.5379797641338541


In [0]:
def get_team_ranking(players):
  result = []
  print('Team:',players)
  if (len(players)==0):
    return blank_rank
  for player in players:
    if (all_results[player][1]>0): # num of competetions
      #print(player, ' rating is ', all_ranking[player])
      result.append(all_ranking[player])
    else:
      result.append(blank_rank)
  return stat.mean(result)


In [21]:
from scipy import stats

spearman = []
pearson = []

test_id = [x for x in test_id if len(results_cleaned2[x])>1] ## фильтруем турниры с 1 командой

for tourn_id in test_id:
  print('Tournament:', tourn_id)
  rating = []
  ranking = []
  i=1
  for team in results_cleaned2[tourn_id]:
    rating.append(get_team_ranking(team[2]))
    ranking.append(-i)
    i+=1
  #print ('Spearman: ',stats.spearmanr(rating, ranking)[0])
  #print ('Pearson: ',stats.pearsonr(rating,ranking)[0])
  # ставим минус т.к. у нас места возрастают а сила убывает
  spearman.append(stats.spearmanr(rating, ranking)[0])
  pearson.append(stats.pearsonr(rating,ranking)[0])

print(spearman)
print(pearson)
print(stat.mean(spearman))
print(stat.mean(pearson))


Streaming output truncated to the last 5000 lines.
Team: [180448, 180449, 197802, 197803, 200990, 200991]
Team: [217110, 217112, 219489, 217111, 217109, 218945]
Team: [135728, 160350, 214301, 214300, 214302, 219090]
Tournament: 6185
Team: [9342, 77673, 10580, 71907, 38307, 43559]
Team: [21026, 15123, 1354, 96642, 66401, 118142]
Team: [126620, 23967, 147750, 28019, 10083, 73380]
Team: [5007, 7737, 29487, 39152, 27436, 24609]
Team: [29578, 6076, 17955, 39879, 173665, 10387]
Team: [3088, 52047, 23486, 1599, 36673, 24844]
Team: [30772, 34417, 52183, 93424, 29425, 13345]
Team: [30371, 61316, 149363, 86251, 103351, 66180]
Team: [1369, 28426, 16703, 7669, 38440, 11491]
Team: [38180, 17780, 13196, 30014, 23996, 27773]
Team: [158734, 3440, 114127, 121186, 112086, 36086]
Team: [26477, 25110, 34684, 34682, 27245, 162306]
Team: [103745, 33706, 22208, 194371, 194328, 195615]
Team: [150165, 137194, 15726, 134619, 208796, 16129]
Team: [102213, 150355, 195793, 39503, 93018, 102548]
Team: [10474, 23804

Теперь будем работать в рамках модели, когда участники команды отвечают на вопрос независимо и если кто то ответил - команда отвечает правильно, если никто - не отвечает. Будем попеременно улучшать оценки для сложностей вопросов и индивидуальных показателей всех участников

In [0]:
from scipy.special import expit
import math

def sigmoid(x):
  return expit(x)

def sigm_reverse(x):
  if x<0.001:
    return -999
  elif x>0.999:
    return 999
  return -math.log(1/x-1)

def adjust_individual(players, team_X, Y):
  prob_real = 1-sigmoid(team_X + Y)
  #print('Prob_real:',prob_real)
  prob_adjusted = 1
  probs = []
  for player in players:
    player_rating = all_ranking[player]
    player_prob = (1 - sigmoid(player_rating + Y))
    probs.append(player_prob)
    prob_adjusted *= (1 - sigmoid(player_rating + Y))
  multiplier = math.pow(prob_real/prob_adjusted, 1.0/len(players))
  #print('Prob_adjusted', prob_adjusted)
  #print(multiplier)
  probs = [x*multiplier for x in probs]
  #print(probs)
  #print([sigm_reverse(1-x) - Y for x in probs])
  return [sigm_reverse(1-x) - Y for x in probs]

def compose_team(players, team_X, Y):
  prob_adjusted = 1
  for player in players:
    player_rating = all_ranking[player]
    #print('player:',player,'rating:',player_rating)
    prob_adjusted *= (1 - sigmoid(player_rating + Y))
  #print('prob_by_member:', prob_adjusted)
  #print('prob_by_condition', 1-sigmoid(team_X+Y))
  return sigm_reverse(1-prob_adjusted) - Y

def get_team_rank(players):
  prob = 1
  for player in players:
    player_rating = all_ranking[player]
    player_prob = (1 - sigmoid(player_rating))
    prob_adjusted *= player_prob
  return sigm_reverse(1-prob)

In [0]:
# creating storage for competetion results

logistic = LogisticRegression(fit_intercept = False)

def record_item(item, value):
  all_results_upd[item][1] += 1
  all_results_upd[item][0] += value

def upd_EM(id, update_questions):
  task3 = []
  words = []
   
  for n in range(len(results_cleaned2[id])): # command id
    for p in range(len(results_cleaned2[id][n][0])): #answers
        quest = str(id)+'_'+str(p)
        team = n
        task3.append((quest, team,results_cleaned2[id][n][0][p]))
        words.append(quest)
        words.append(team)

  if len(words)==0:
    return 
  
  words = set(words) # encoding questions and teams no

  idx_to_word = dict(enumerate(words))
  word_to_idx = dict([(y,x) for x,y in idx_to_word.items()])
  #preparing matrix
  X = np.zeros((len(task3),len(words)))
  Y = np.zeros(len(task3))
  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = 1 if str(task3[i][2]) == '1' else 0

  if len(set(Y)) <= 1:
    print ('<2 categories , skipping')
    return

  logistic.fit(X, Y)

  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = 1 if str(task3[i][2]) == '1' else 0


  for i in range(len(task3)):
    question = word_to_idx[task3[i][0]]
    team = word_to_idx[task3[i][1]]
    k_quest = logistic.coef_[0][question]
    k_team = logistic.coef_[0][team]
    players_list = results_cleaned2[id][task3[i][1]][2]
    if (update_questions):
      K = compose_team(players_list, k_team,all_ranking[task3[i][0]])
      record_item(task3[i][0], k_quest + k_team - K)
    else: # team members
      #print(players_list, k_team, task3[i][0],all_ranking[task3[i][0]])
      ranking_split = adjust_individual(players_list, k_team,all_ranking[task3[i][0]])
      for j in range(len(players_list)):
        record_item(players_list[j], ranking_split[j])

In [35]:
import tqdm

all_results_upd = {}
for s in all_words:
  all_results_upd[s] = [0,0] # total rating and num of competetions

def record_new_ratings():
  global blank_rank
  for k,v in all_results_upd.items():
    if v[1] != 0:
      all_ranking[k] = v[0]/v[1] 
  print('blank_rank before',blank_rank)
  blank_rank = stat.mean ([ v for k,v in all_ranking.items() if is_player(k)])
  print('blank_rank after',blank_rank)

  all_results_upd = {}
  for s in all_words:
    all_results_upd[s] = [0,0] # total rating and num of competetions


def calc_corellations():
  spearman = []
  pearson = []

  for tourn_id in test_id:
    #print('Tournament:', tourn_id)
    rating = []
    ranking = []
    i=1
    for team in results_cleaned2[tourn_id]:
      rating.append(get_team_rank(team[2]))
      ranking.append(i)
      i+=1
    #print ('Spearman: ',-stats.spearmanr(rating, ranking)[0])
    #print ('Pearson: ',-stats.pearsonr(rating,ranking)[0])
    # ставим минус т.к. у нас места возрастают а сила убывает
    spearman.append(-stats.spearmanr(rating, ranking)[0])
    pearson.append(-stats.pearsonr(rating,ranking)[0])
  
  print('Spearman: ',stat.mean(spearman))
  print('Pearson: ',stat.mean(pearson))  


def EM_cycle():  
  print('Updating individual ranking')
  for tourn_id in tqdm.tqdm(train_id):
    upd_EM(tourn_id, update_questions = False)

  record_new_ratings()
  print('Updating questions')
  for tourn_id in tqdm.tqdm(train_id):
    upd_EM(tourn_id, update_questions = True)

  calc_corellations()

EM_cycle()

  0%|          | 0/675 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
[-2.310596224034317, -2.3075754686030705, -2.081289683919653]
[20005, 171960, 117733] -1.0436400926851415 4772_3 0.10551412582119211
Prob_real: 0.7187209560717254
Prob_adjusted 0.10353888165464445
1.907592528393712
[0.9024943927720139, 0.9022562108813319, 0.8826446951293444]
[-2.330766716633139, -2.3280629946713542, -2.1232307348552366]
[20005, 171960, 117733] -1.0436400926851415 4772_4 2.1125938895456775
Prob_real: 0.2556020939889182
Prob_adjusted 0.0012018906717569395
5.968975747317716
[0.642683791297002, 0.642396591330355, 0.6191043173257436]
[-2.6996255934935975, -2.698375170688007, -2.5983421347163187]
[20005, 171960, 117733] -1.0436400926851415 4772_5 1.6668848666850162
Prob_real: 0.34904383841676745
Prob_adjusted 0.003845920402541554
4.493932671019671
[0.7125177412521598, 0.712217481994819, 0.6878150333695098]
[-2.574528528890673, -2.5730631346414885, -2.456808967713307]
[20005, 171960, 117733] -1.0436400926851415 4772_6 1.66688

  0%|          | 1/675 [00:10<2:02:01, 10.86s/it]

Streaming output truncated to the last 5000 lines.
[-4.844580184914699, -1.8734606761280614, -1.6907690108014626, -998.6055130821932, -998.6055130821932, -998.6055130821932]
[156965, 156966, 169313, 154988, 183620, 189647] -1.777834714215748 4973_12 1.577311613291824
Prob_real: 0.5499634700822743
Prob_adjusted 0.00812998510317674
2.0185554418556144
[0.9303369124746357, 0.6760455474291962, 0.6391371425358191, 1.110131755356372, 1.110131755356372, 1.110131755356372]
[-4.169187818569817, -2.312969137493768, -2.148932675265574, -1000.5773116132918, -1000.5773116132918, -1000.5773116132918]
[156965, 156966, 169313, 154988, 183620, 189647] -1.777834714215748 4973_13 0.5560847316849843
Prob_real: 0.7723713672138084
Prob_adjusted 0.10634703682031864
1.3916076110215936
[0.9791282348604103, 0.8113377436076995, 0.7829896630143327, 1.07483787314989, 1.07483787314989, 1.07483787314989]
[-4.40435005583884, -2.0148107391355556, -1.839259237534395, -999.556084731685, -999.556084731685, -999.5560847316

  0%|          | 2/675 [01:00<4:11:38, 22.44s/it]

Streaming output truncated to the last 5000 lines.
5.076223475889059
[0.42966222246949604, 0.6381321290747902, 0.6116627928865906, 0.6677367227604616]
[-1.9876563783138088, -2.838151997942836, -2.725192870894361, -2.9688520947857087]
[66315, 20946, 22265, 59480] -0.2002497731368229 4974_17 -1.2701451800967198
Prob_real: 0.8131174095552672
Prob_adjusted 0.43899230648363824
1.1666056828552382
[0.8882063640433449, 0.9708975692194103, 0.9629021291329245, 0.9792279892895698]
[-0.8024042922011556, -2.2372540899719624, -1.986247016375341, -2.583012873485563]
[66315, 20946, 22265, 59480] -0.2002497731368229 4974_18 -0.8076322590652195
Prob_real: 0.7326054550220915
Prob_adjusted 0.2903466721658703
1.260341752757331
[0.8414837414617276, 0.9547232064361165, 0.9434001723152559, 0.9666093428535175]
[-0.8616772678927408, -2.2409945853432784, -2.005852353673012, -2.557886031287753]
[66315, 20946, 22265, 59480] -0.2002497731368229 4974_19 -0.27833319071299273
Prob_real: 0.6174132065328846
Prob_adjuste

KeyboardInterrupt: ignored